# Section 11: Audit API - Comprehensive Testing

## 📋 Overview
This notebook provides comprehensive testing for **Section 11: Audit** of the VeevaVault API.

Audit API enables retrieval of audit trails, access logs, user activity tracking, system events monitoring, compliance reporting, and security audit information within the Vault ecosystem.

## 🎯 Endpoints to Test

### Audit Trail Retrieval (AuditService)
1. **Retrieve Audit Trail** - `GET /api/{version}/audittrail`
2. **Retrieve Document Audit** - `GET /api/{version}/objects/documents/{document_id}/audittrail`
3. **Retrieve Object Audit** - `GET /api/{version}/objects/{object_name}/{object_id}/audittrail`
4. **Retrieve User Audit** - `GET /api/{version}/objects/users/{user_id}/audittrail`

### Activity Logs (ActivityLogService)
5. **Retrieve Activity Logs** - `GET /api/{version}/logs/activity`
6. **Retrieve Login History** - `GET /api/{version}/logs/login`
7. **Retrieve API Usage Logs** - `GET /api/{version}/logs/api`
8. **Retrieve System Events** - `GET /api/{version}/logs/system`

### Security Audit (SecurityAuditService)
9. **Retrieve Security Events** - `GET /api/{version}/logs/security`
10. **Retrieve Permission Changes** - `GET /api/{version}/logs/permissions`
11. **Retrieve Role Changes** - `GET /api/{version}/logs/roles`
12. **Retrieve Access Violations** - `GET /api/{version}/logs/violations`

### Compliance Reporting (ComplianceService)
13. **Generate Compliance Report** - `POST /api/{version}/reports/compliance`
14. **Retrieve Compliance Status** - `GET /api/{version}/compliance/status`
15. **Retrieve Regulatory Events** - `GET /api/{version}/logs/regulatory`
16. **Export Audit Data** - `POST /api/{version}/export/audit`

### Audit Configuration
17. **Retrieve Audit Settings** - `GET /api/{version}/settings/audit`
18. **Audit Policy Information** - Audit retention and policy details
19. **Audit Categories** - Available audit event categories
20. **Audit Filters** - Filtering and search capabilities

### Data Privacy and GDPR
21. **Data Subject Requests** - Privacy-related audit queries
22. **Data Processing Logs** - GDPR compliance tracking
23. **Consent Management Audit** - Consent tracking and audit
24. **Data Retention Audit** - Data lifecycle and retention tracking

### 🔬 Testing Strategy
- **Discovery First**: Query audit metadata to understand available audit categories and retention
- **Safe Operations**: Use read-only audit retrieval, no audit configuration changes
- **Intelligent Testing**: Use existing data for audit trail testing with proper filtering
- **Non-Destructive**: Ensure audit tests don't impact audit configuration or data retention

---

In [1]:
# Essential imports for comprehensive Audit API testing
import sys
import time
import json
import requests
import tempfile
import os
from typing import Dict, List, Any, Optional
from datetime import datetime, timezone, timedelta

# VeevaVault imports
sys.path.append('/Users/mp/Documents/Code/VeevaTools/veevatools')
from veevavault.client import VaultClient
from veevavault.services.authentication import AuthenticationService

print("✅ Essential imports loaded for Audit API testing")

✅ Essential imports loaded for Audit API testing


## 🔐 Load Test Credentials

Loading secure credentials for VeevaVault API testing.

In [2]:
# Import credentials from secure configuration
try:
    from test_credentials import TEST_VAULTS, DEFAULT_VAULT
    
    # Get the default vault credentials
    vault_config = TEST_VAULTS[DEFAULT_VAULT]
    VAULT_URL = vault_config["URL"]
    VAULT_USERNAME = vault_config["username"] 
    VAULT_PASSWORD = vault_config["password"]
    VAULT_DNS = VAULT_URL.replace("https://", "").replace("http://", "").rstrip("/")
    
    print("✅ Successfully imported credentials from test_credentials.py")
    print(f"🏢 Using vault: {DEFAULT_VAULT}")
    print(f"📡 Vault DNS: {VAULT_DNS}")
    print(f"👤 Username: {VAULT_USERNAME}")
    print(f"🔗 Full URL: {VAULT_URL}")
    
except ImportError as e:
    print("❌ Failed to import credentials!")
    print("Please ensure test_credentials.py exists and contains:")
    print("- TEST_VAULTS dictionary")
    print("- DEFAULT_VAULT setting")
    raise e
except KeyError as e:
    print(f"❌ Configuration error: {e}")
    print("Please check your test_credentials.py configuration")
    raise e

✅ Successfully imported credentials from test_credentials.py
🏢 Using vault: michael_mastermind
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔗 Full URL: https://vv-consulting-michael-mastermind.veevavault.com/


## 🚀 Initialize Enhanced Framework

Setting up the enhanced baseline API testing framework with Audit-specific functionality.

In [3]:
# Enhanced baseline API testing framework for Audit operations
from dataclasses import dataclass
from typing import Dict, List, Any, Optional
import time
import uuid

@dataclass
class APITestResult:
    """Standard result structure for API tests"""
    endpoint: str
    method: str
    success: bool
    response_time: float
    status_code: Optional[int] = None
    response_data: Optional[Dict] = None
    error_message: Optional[str] = None
    test_name: Optional[str] = None
    details: Optional[Dict] = None

class EnhancedAuditAPITester:
    """Enhanced baseline API testing framework with Audit-specific functionality"""
    
    def __init__(self, vault_url: str, username: str, password: str):
        self.vault_url = vault_url.rstrip('/')
        self.vault_dns = vault_url.replace("https://", "").replace("http://", "").rstrip("/")
        self.username = username
        self.password = password
        self.client = None
        self.session_id = None
        self.vault_id = None
        self.results = []
        self.api_version = "v25.2"
        
        # Audit-specific tracking
        self.discovered_audit_trails = []
        self.discovered_audit_types = []
        self.discovered_activity_logs = []     # Track activity log entries
        self.discovered_security_events = []   # Track security audit events
        self.audit_date_range = {}             # Track audit data date ranges
        self.audit_categories = []             # Track available audit categories
        
        # Initialize VaultClient
        print("🔧 Initializing VeevaVault client...")
        self.client = VaultClient()
        self.client.vaultDNS = self.vault_dns
        self.client.vaultURL = self.vault_url
        self.client.vaultUserName = self.username
        self.client.vaultPassword = self.password
        print("✅ VeevaVault client initialized")
        print(f"🔗 Vault URL: {self.vault_url}")
        print(f"📡 Vault DNS: {self.vault_dns}")
        print(f"👤 Username: {self.username}")
        
        # Set up enhanced framework
        print("🔧 Initializing enhanced test framework...")
        print("✅ Enhanced baseline framework ready")
    
    def authenticate(self) -> bool:
        """Authenticate with the vault"""
        try:
            print("🔐 Performing authentication...")
            start_time = time.time()
            
            auth_service = AuthenticationService(self.client)
            response = auth_service.authenticate_with_username_password(
                username=self.username,
                password=self.password
            )
            
            response_time = time.time() - start_time
            
            if response and isinstance(response, dict) and response.get('responseStatus') == 'SUCCESS':
                self.session_id = response.get('sessionId')
                self.vault_id = response.get('vaultId')
                
                # Update client with session
                self.client.sessionId = self.session_id
                self.client.vaultId = self.vault_id
                
                result = APITestResult(
                    endpoint='/api/{version}/auth',
                    method='POST',
                    success=True,
                    response_time=response_time,
                    test_name='Authentication',
                    details={'vault_id': self.vault_id, 'has_session': bool(self.session_id)}
                )
                self.results.append(result)
                
                print(f"✅ Authentication [01]: SUCCESS ({response_time:.2f}s)")
                print(f"\\n🎉 Authentication Successful!")
                print(f"🆔 Session ID: {self.session_id[:20]}...")
                print(f"🏢 Vault ID: {self.vault_id}")
                return True
                
        except Exception as e:
            print(f"❌ Authentication failed: {str(e)}")
            return False
            
        return False
    
    def make_authenticated_request(self, endpoint: str, method: str = 'GET', **kwargs) -> APITestResult:
        """Make an authenticated API request"""
        url = f"{self.vault_url}{endpoint}"
        headers = {
            'Authorization': self.session_id,
            'Accept': 'application/json'
        }
        headers.update(kwargs.get('headers', {}))
        
        start_time = time.time()
        try:
            response = requests.request(method, url, headers=headers, **{k: v for k, v in kwargs.items() if k != 'headers'})
            response_time = time.time() - start_time
            
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=response.status_code < 400,
                response_time=response_time,
                status_code=response.status_code,
                response_data=response.json() if response.content and response.headers.get('content-type', '').startswith('application/json') else None
            )
        except Exception as e:
            response_time = time.time() - start_time
            return APITestResult(
                endpoint=endpoint,
                method=method,
                success=False,
                response_time=response_time,
                error_message=str(e)
            )
    
    def print_summary(self):
        """Print test summary"""
        if not self.results:
            print("📊 No tests completed yet")
            return
            
        successful_tests = len([r for r in self.results if r.success])
        total_time = sum(r.response_time for r in self.results)
        
        print(f"\\n📊 TEST SUMMARY")
        print("=" * 40)
        print(f"✅ SUCCESS: {successful_tests}")
        if successful_tests < len(self.results):
            print(f"❌ FAILED: {len(self.results) - successful_tests}")
        print(f"⏱️ Total time: {total_time:.2f}s")
        print(f"🔐 Session: {'✅ Active' if self.session_id else '❌ None'}")

# Initialize the enhanced audit tester
audit_tester = EnhancedAuditAPITester(VAULT_URL, VAULT_USERNAME, VAULT_PASSWORD)

print("✅ Enhanced Audit API testing framework initialized")

🔧 Initializing VeevaVault client...
✅ VeevaVault client initialized
🔗 Vault URL: https://vv-consulting-michael-mastermind.veevavault.com
📡 Vault DNS: vv-consulting-michael-mastermind.veevavault.com
👤 Username: veevatools@vv-consulting.com
🔧 Initializing enhanced test framework...
✅ Enhanced baseline framework ready
✅ Enhanced Audit API testing framework initialized


## 🔐 Authentication & Service Initialization

Testing authentication for comprehensive audit operations.

In [4]:
# Test 01: Authentication
print("🔐 Testing authentication...")
print("=" * 60)

auth_success = audit_tester.authenticate()

if auth_success:
    print(f"\\n✅ Session established: {audit_tester.session_id[:20]}...")
    print(f"✅ Vault ID: {audit_tester.vault_id}")
    
    # Verify we can access audit-related endpoints
    print("\\n📋 Audit API access preparation:")
    print(f"  • Session Authorization: Available")
    print(f"  • Vault Context: Established")
    print(f"  • API Version: {audit_tester.api_version}")
    
else:
    print("❌ Authentication failed - cannot proceed with audit tests")
    
print("\\n" + "=" * 60)

🔐 Testing authentication...
🔐 Performing authentication...
✅ Authentication [01]: SUCCESS (0.42s)
\n🎉 Authentication Successful!
🆔 Session ID: 8B4CF009421E159A51A1...
🏢 Vault ID: 92425
\n✅ Session established: 8B4CF009421E159A51A1...
✅ Vault ID: 92425
\n📋 Audit API access preparation:
  • Session Authorization: Available
  • Vault Context: Established
  • API Version: v25.2
\n============================================================


## 🔍 Audit Trail Discovery

Discovering available audit trails and audit information in the vault.

In [5]:
# Test 02: Retrieve general audit trail
print("📋 Testing general audit trail retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve general audit trail with basic parameters
    endpoint = f"/api/{audit_tester.api_version}/audittrail"
    audit_params = {
        'params': {
            'limit': 10,  # Limit to reduce response size
            'start_date': (datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d')  # Last 30 days
        }
    }
    
    audit_result = audit_tester.make_authenticated_request(endpoint, **audit_params)
    response_time = time.time() - start_time
    
    if audit_result.success and audit_result.response_data:
        audit_data = audit_result.response_data.get('data', [])
        audit_tester.discovered_audit_trails = audit_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve General Audit Trail',
            details={
                'total_audit_entries': len(audit_data) if isinstance(audit_data, list) else 1,
                'audit_type': type(audit_data).__name__,
                'has_audit_data': bool(audit_data),
                'sample_event_types': [entry.get('event_type') for entry in audit_data[:3] if isinstance(entry, dict)] if isinstance(audit_data, list) else []
            }
        )
        audit_tester.results.append(result)
        
        print(f"✅ Retrieve General Audit Trail [02]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(audit_data, list) and audit_data:
            print(f"📋 Found {len(audit_data)} audit entries")
            
            # Analyze event types
            event_types = [entry.get('event_type', entry.get('action', 'Unknown')) for entry in audit_data if isinstance(entry, dict)]
            unique_event_types = list(set(event_types))
            audit_tester.discovered_audit_types = unique_event_types
            
            print(f"\\n🔧 Discovered Event Types: {len(unique_event_types)}")
            for i, event_type in enumerate(unique_event_types[:10]):
                count = event_types.count(event_type)
                print(f"  {i+1}. {event_type} ({count} entries)")
            if len(unique_event_types) > 10:
                print(f"  ... and {len(unique_event_types) - 10} more types")
            
            # Show sample audit entries
            print("\\n📋 Sample Audit Entries:")
            for i, entry in enumerate(audit_data[:3]):
                if isinstance(entry, dict):
                    event_type = entry.get('event_type', entry.get('action', 'Unknown'))
                    timestamp = entry.get('timestamp', entry.get('date_created', 'Unknown'))
                    user = entry.get('user_name', entry.get('user', 'Unknown'))
                    object_type = entry.get('object_type', entry.get('item', 'Unknown'))
                    print(f"  {i+1}. Event: {event_type}")
                    print(f"     Time: {timestamp}, User: {user}")
                    print(f"     Object: {object_type}")
                else:
                    print(f"  {i+1}. {entry}")
        elif isinstance(audit_data, dict):
            print("📋 Retrieved audit trail structure")
            if 'audit_trail' in audit_data:
                audit_list = audit_data['audit_trail']
                print(f"📊 Contains {len(audit_list) if isinstance(audit_list, list) else 'N/A'} audit entries")
        else:
            print("🔍 No audit entries found or unexpected format")
        
    else:
        error_msg = f"API returned {audit_result.status_code}" if audit_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve General Audit Trail',
            error_message=error_msg
        )
        audit_tester.results.append(result)
        print(f"❌ Retrieve General Audit Trail [02]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {audit_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/audittrail',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve General Audit Trail',
        error_message=str(e)
    )
    audit_tester.results.append(result)
    print(f"❌ Retrieve General Audit Trail [02]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

📋 Testing general audit trail retrieval...
❌ Retrieve General Audit Trail [02]: FAILED (0.20s)
   Status Code: 404
   Error: API returned 404
\n============================================================


In [6]:
# Test 03: Retrieve activity logs
print("📊 Testing activity logs retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve activity logs
    endpoint = f"/api/{audit_tester.api_version}/logs/activity"
    activity_params = {
        'params': {
            'limit': 10,  # Limit to reduce response size
            'start_date': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')  # Last 7 days
        }
    }
    
    activity_result = audit_tester.make_authenticated_request(endpoint, **activity_params)
    response_time = time.time() - start_time
    
    if activity_result.success and activity_result.response_data:
        activity_data = activity_result.response_data.get('data', [])
        audit_tester.discovered_activity_logs = activity_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Activity Logs',
            details={
                'total_activity_entries': len(activity_data) if isinstance(activity_data, list) else 1,
                'activity_type': type(activity_data).__name__,
                'has_activity_data': bool(activity_data),
                'sample_activities': [entry.get('activity') for entry in activity_data[:3] if isinstance(entry, dict)] if isinstance(activity_data, list) else []
            }
        )
        audit_tester.results.append(result)
        
        print(f"✅ Retrieve Activity Logs [03]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(activity_data, list) and activity_data:
            print(f"📊 Found {len(activity_data)} activity log entries")
            
            # Show sample activity entries
            print("\\n📋 Sample Activity Entries:")
            for i, entry in enumerate(activity_data[:3]):
                if isinstance(entry, dict):
                    activity = entry.get('activity', entry.get('action', 'Unknown'))
                    timestamp = entry.get('timestamp', entry.get('date_created', 'Unknown'))
                    user = entry.get('user_name', entry.get('user', 'Unknown'))
                    ip_address = entry.get('ip_address', entry.get('ip', 'Unknown'))
                    print(f"  {i+1}. Activity: {activity}")
                    print(f"     Time: {timestamp}, User: {user}")
                    print(f"     IP: {ip_address}")
                else:
                    print(f"  {i+1}. {entry}")
                    
            if len(activity_data) > 3:
                print(f"  ... and {len(activity_data) - 3} more entries")
        elif isinstance(activity_data, dict):
            print("📊 Retrieved activity logs structure")
        else:
            print("🔍 No activity log entries found or unexpected format")
        
    else:
        error_msg = f"API returned {activity_result.status_code}" if activity_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Activity Logs',
            error_message=error_msg
        )
        audit_tester.results.append(result)
        print(f"❌ Retrieve Activity Logs [03]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {activity_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/logs/activity',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Activity Logs',
        error_message=str(e)
    )
    audit_tester.results.append(result)
    print(f"❌ Retrieve Activity Logs [03]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

📊 Testing activity logs retrieval...
❌ Retrieve Activity Logs [03]: FAILED (0.26s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 🔒 Security & Login Logs

Testing security event logs and login history retrieval.

In [7]:
# Test 04: Retrieve login history logs
print("🔑 Testing login history retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve login history
    endpoint = f"/api/{audit_tester.api_version}/logs/login"
    login_params = {
        'params': {
            'limit': 10,  # Limit to reduce response size
            'start_date': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')  # Last 7 days
        }
    }
    
    login_result = audit_tester.make_authenticated_request(endpoint, **login_params)
    response_time = time.time() - start_time
    
    if login_result.success and login_result.response_data:
        login_data = login_result.response_data.get('data', [])
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Login History',
            details={
                'total_login_entries': len(login_data) if isinstance(login_data, list) else 1,
                'login_type': type(login_data).__name__,
                'has_login_data': bool(login_data),
                'sample_login_results': [entry.get('result') for entry in login_data[:3] if isinstance(entry, dict)] if isinstance(login_data, list) else []
            }
        )
        audit_tester.results.append(result)
        
        print(f"✅ Retrieve Login History [04]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(login_data, list) and login_data:
            print(f"🔑 Found {len(login_data)} login history entries")
            
            # Analyze login results
            login_results = [entry.get('result', entry.get('status', 'Unknown')) for entry in login_data if isinstance(entry, dict)]
            successful_logins = [r for r in login_results if r.lower() in ['success', 'successful', 'ok']]
            failed_logins = [r for r in login_results if r.lower() in ['failed', 'failure', 'error']]
            
            print(f"\\n📊 Login Analysis:")
            print(f"  ✅ Successful: {len(successful_logins)}")
            print(f"  ❌ Failed: {len(failed_logins)}")
            print(f"  ❓ Other: {len(login_results) - len(successful_logins) - len(failed_logins)}")
            
            # Show sample login entries
            print("\\n📋 Sample Login Entries:")
            for i, entry in enumerate(login_data[:3]):
                if isinstance(entry, dict):
                    result_status = entry.get('result', entry.get('status', 'Unknown'))
                    timestamp = entry.get('timestamp', entry.get('date_created', 'Unknown'))
                    user = entry.get('user_name', entry.get('user', 'Unknown'))
                    ip_address = entry.get('ip_address', entry.get('ip', 'Unknown'))
                    print(f"  {i+1}. Result: {result_status}")
                    print(f"     Time: {timestamp}, User: {user}")
                    print(f"     IP: {ip_address}")
                else:
                    print(f"  {i+1}. {entry}")
                    
            if len(login_data) > 3:
                print(f"  ... and {len(login_data) - 3} more entries")
        elif isinstance(login_data, dict):
            print("🔑 Retrieved login history structure")
        else:
            print("🔍 No login history entries found or unexpected format")
        
    else:
        error_msg = f"API returned {login_result.status_code}" if login_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Login History',
            error_message=error_msg
        )
        audit_tester.results.append(result)
        print(f"❌ Retrieve Login History [04]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {login_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/logs/login',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Login History',
        error_message=str(e)
    )
    audit_tester.results.append(result)
    print(f"❌ Retrieve Login History [04]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

🔑 Testing login history retrieval...
❌ Retrieve Login History [04]: FAILED (0.18s)
   Status Code: 404
   Error: API returned 404
\n============================================================


In [8]:
# Test 05: Retrieve security events
print("🔒 Testing security events retrieval...")
print("=" * 60)

try:
    start_time = time.time()
    
    # Attempt to retrieve security events
    endpoint = f"/api/{audit_tester.api_version}/logs/security"
    security_params = {
        'params': {
            'limit': 10,  # Limit to reduce response size
            'start_date': (datetime.now() - timedelta(days=7)).strftime('%Y-%m-%d')  # Last 7 days
        }
    }
    
    security_result = audit_tester.make_authenticated_request(endpoint, **security_params)
    response_time = time.time() - start_time
    
    if security_result.success and security_result.response_data:
        security_data = security_result.response_data.get('data', [])
        audit_tester.discovered_security_events = security_data
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=True,
            response_time=response_time,
            test_name='Retrieve Security Events',
            details={
                'total_security_entries': len(security_data) if isinstance(security_data, list) else 1,
                'security_type': type(security_data).__name__,
                'has_security_data': bool(security_data),
                'sample_event_types': [entry.get('event_type') for entry in security_data[:3] if isinstance(entry, dict)] if isinstance(security_data, list) else []
            }
        )
        audit_tester.results.append(result)
        
        print(f"✅ Retrieve Security Events [05]: SUCCESS ({response_time:.2f}s)")
        
        if isinstance(security_data, list) and security_data:
            print(f"🔒 Found {len(security_data)} security event entries")
            
            # Analyze security event types
            event_types = [entry.get('event_type', entry.get('security_event', 'Unknown')) for entry in security_data if isinstance(entry, dict)]
            unique_event_types = list(set(event_types))
            
            print(f"\\n🔧 Security Event Types: {len(unique_event_types)}")
            for i, event_type in enumerate(unique_event_types[:5]):
                count = event_types.count(event_type)
                print(f"  {i+1}. {event_type} ({count} events)")
            if len(unique_event_types) > 5:
                print(f"  ... and {len(unique_event_types) - 5} more types")
            
            # Show sample security entries
            print("\\n📋 Sample Security Events:")
            for i, entry in enumerate(security_data[:3]):
                if isinstance(entry, dict):
                    event_type = entry.get('event_type', entry.get('security_event', 'Unknown'))
                    timestamp = entry.get('timestamp', entry.get('date_created', 'Unknown'))
                    user = entry.get('user_name', entry.get('user', 'Unknown'))
                    severity = entry.get('severity', entry.get('level', 'Unknown'))
                    print(f"  {i+1}. Event: {event_type}")
                    print(f"     Time: {timestamp}, User: {user}")
                    print(f"     Severity: {severity}")
                else:
                    print(f"  {i+1}. {entry}")
                    
            if len(security_data) > 3:
                print(f"  ... and {len(security_data) - 3} more entries")
        elif isinstance(security_data, dict):
            print("🔒 Retrieved security events structure")
        else:
            print("🔍 No security event entries found or unexpected format")
        
    else:
        error_msg = f"API returned {security_result.status_code}" if security_result.status_code else "Unknown error"
        
        result = APITestResult(
            endpoint=endpoint,
            method='GET',
            success=False,
            response_time=response_time,
            test_name='Retrieve Security Events',
            error_message=error_msg
        )
        audit_tester.results.append(result)
        print(f"❌ Retrieve Security Events [05]: FAILED ({response_time:.2f}s)")
        print(f"   Status Code: {security_result.status_code}")
        print(f"   Error: {error_msg}")
        
except Exception as e:
    response_time = time.time() - start_time if 'start_time' in locals() else 0
    result = APITestResult(
        endpoint='/api/{version}/logs/security',
        method='GET',
        success=False,
        response_time=response_time,
        test_name='Retrieve Security Events',
        error_message=str(e)
    )
    audit_tester.results.append(result)
    print(f"❌ Retrieve Security Events [05]: EXCEPTION ({response_time:.2f}s)")
    print(f"   Exception: {str(e)}")

print("\\n" + "=" * 60)

🔒 Testing security events retrieval...
❌ Retrieve Security Events [05]: FAILED (0.17s)
   Status Code: 404
   Error: API returned 404
\n============================================================


## 🎯 Test Summary & Cleanup

Comprehensive results analysis for audit testing (all operations are read-only and safe).

In [9]:
# Comprehensive Test Summary and Cleanup
print("🎯 AUDIT API TESTING - FINAL SUMMARY")
print("=" * 60)

# Print comprehensive summary
audit_tester.print_summary()

# Detailed results analysis
print("\\n📊 DETAILED RESULTS")
print("-" * 40)

success_count = 0
total_time = 0

for i, result in enumerate(audit_tester.results, 1):
    status = "✅ PASS" if result.success else "❌ FAIL"
    test_name = result.test_name or f"Test {i}"
    
    print(f"{status} [{i:02d}]: {test_name} ({result.response_time:.2f}s)")
    
    if result.success:
        success_count += 1
        
    total_time += result.response_time
    
    # Show additional details for interesting results
    if result.details:
        if 'total_audit_entries' in result.details:
            print(f"        📋 Audit Entries: {result.details['total_audit_entries']}")
        if 'total_activity_entries' in result.details:
            print(f"        📊 Activity Entries: {result.details['total_activity_entries']}")
        if 'total_login_entries' in result.details:
            print(f"        🔑 Login Entries: {result.details['total_login_entries']}")
        if 'total_security_entries' in result.details:
            print(f"        🔒 Security Entries: {result.details['total_security_entries']}")
        if 'sample_event_types' in result.details and result.details['sample_event_types']:
            print(f"        🔧 Event Types: {', '.join(result.details['sample_event_types'][:3])}")
    
    if not result.success and result.error_message:
        print(f"        ❌ Error: {result.error_message}")

# Final statistics
print("\\n🏆 FINAL STATISTICS")
print("-" * 40)
print(f"📈 Success Rate: {success_count}/{len(audit_tester.results)} ({(success_count/len(audit_tester.results)*100):.1f}%)")
print(f"⏱️ Total Execution Time: {total_time:.2f}s")
print(f"📋 Audit Entries Discovered: {len(audit_tester.discovered_audit_trails)}")
print(f"📊 Activity Log Entries: {len(audit_tester.discovered_activity_logs)}")
print(f"🔒 Security Events: {len(audit_tester.discovered_security_events)}")
print(f"🔧 Audit Types Discovered: {len(audit_tester.discovered_audit_types)}")

# Vault state summary
print("\\n🏢 VAULT STATE")
print("-" * 40)
print(f"🔗 Connected to: {audit_tester.vault_url}")
print(f"🆔 Vault ID: {audit_tester.vault_id}")
print(f"👤 User: {audit_tester.username}")
print(f"🔐 Session: {'Active' if audit_tester.session_id else 'None'}")

# API coverage summary
tested_endpoints = [
    "Authentication",
    "Retrieve General Audit Trail", 
    "Retrieve Activity Logs",
    "Retrieve Login History",
    "Retrieve Security Events"
]

print("\\n🎯 API COVERAGE")
print("-" * 40)
for i, endpoint in enumerate(tested_endpoints, 1):
    status = "✅" if i <= len(audit_tester.results) else "⏸️"
    print(f"{status} {endpoint}")

print("\\n✅ Audit API testing completed!")
print("📋 All operations were read-only and audit-safe")
print("🔍 No audit configuration modified - vault integrity preserved")

# Export summary for next section
audit_summary = {
    'section': '11_audit',
    'total_tests': len(audit_tester.results),
    'successful_tests': success_count,
    'total_time': total_time,
    'success_rate': (success_count/len(audit_tester.results)*100) if audit_tester.results else 0,
    'discovered_audit_entries': len(audit_tester.discovered_audit_trails),
    'discovered_activity_logs': len(audit_tester.discovered_activity_logs),
    'discovered_security_events': len(audit_tester.discovered_security_events),
    'audit_types_discovered': len(audit_tester.discovered_audit_types),
    'vault_id': audit_tester.vault_id,
    'session_active': bool(audit_tester.session_id)
}

print(f"\\n📦 Section summary exported for continuation: {audit_summary['success_rate']:.1f}% success rate")
print("=" * 60)

🎯 AUDIT API TESTING - FINAL SUMMARY
\n📊 TEST SUMMARY
✅ SUCCESS: 1
❌ FAILED: 4
⏱️ Total time: 1.23s
🔐 Session: ✅ Active
\n📊 DETAILED RESULTS
----------------------------------------
✅ PASS [01]: Authentication (0.42s)
❌ FAIL [02]: Retrieve General Audit Trail (0.20s)
        ❌ Error: API returned 404
❌ FAIL [03]: Retrieve Activity Logs (0.26s)
        ❌ Error: API returned 404
❌ FAIL [04]: Retrieve Login History (0.18s)
        ❌ Error: API returned 404
❌ FAIL [05]: Retrieve Security Events (0.17s)
        ❌ Error: API returned 404
\n🏆 FINAL STATISTICS
----------------------------------------
📈 Success Rate: 1/5 (20.0%)
⏱️ Total Execution Time: 1.23s
📋 Audit Entries Discovered: 0
📊 Activity Log Entries: 0
🔒 Security Events: 0
🔧 Audit Types Discovered: 0
\n🏢 VAULT STATE
----------------------------------------
🔗 Connected to: https://vv-consulting-michael-mastermind.veevavault.com
🆔 Vault ID: 92425
👤 User: veevatools@vv-consulting.com
🔐 Session: Active
\n🎯 API COVERAGE
------------------